In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')
import sys
sys.path.append('/home/bnijholt/orbitalfield/')

In [ ]:
import os
from scripts.hpc05 import HPC05Client
os.environ['SSH_AUTH_SOCK'] = os.path.join(os.path.expanduser('~'), 'ssh-agent.socket')
client = HPC05Client()

In [ ]:
# from ipyparallel import Client
# client = Client(profile='python3')
dview = client[:]
dview.use_dill()
lview = client.load_balanced_view()
len(dview)

In [ ]:
%%px --local

import kwant
from kwant.digest import uniform
from types import SimpleNamespace
import sympy
from sympy.physics.quantum import TensorProduct as kr
import numpy as np
from scipy.constants import hbar, m_e, eV, physical_constants
from discretizer import Discretizer, momentum_operators
import scipy.optimize
from itertools import product
from copy import copy
import types
import os.path
import deepdish as dd
import matplotlib.pyplot as plt

k_B = physical_constants['Boltzmann constant in eV/K'][0] * 1000
sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)
s0 = sympy.eye(2)
s0sz = np.kron(s0, sz)
s0s0 = np.kron(s0, s0)

# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = SimpleNamespace(
    m=0.015 * m_e,  # effective mass in kg
    g=50,  # Lande factor
    hbar=hbar,
    m_e=m_e,
    e=eV,
    eV=eV,
    meV=eV * 1e-3)

constants.t = (hbar ** 2 / (2 * constants.m)) * (1e18 / constants.meV)  # meV * nm^2
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV

class SimpleNamespace(SimpleNamespace):
    def update(self, **kwargs):
        self.__dict__.update(kwargs)
        return self

def make_params(alpha=20,
                B_x=0,
                B_y=0,
                B_z=0,
                Delta=0.25,
                mu=0,
                orbital=True,
                t=constants.t,
                g=constants.g,
                mu_B=constants.mu_B,
                V=lambda x,y,z: 0,
                **kwargs):
    """Function that creates a namespace with parameters.

    Parameters:
    -----------
    alpha : float
        Spin-orbit coupling strength in units of meV*nm.
    B_x, B_y, B_z : float
        The magnetic field strength in the x, y and z direction in units of Tesla.
    Delta : float
        The superconducting gap in units of meV.
    mu : float
        The chemical potential in units of meV.
    orbital : bool
        Switches the orbital effects on and off.
    t : float
        Hopping parameter in meV * nm^2.
    g : float
        Lande g factor.
    mu_B : float
        Bohr magneton in meV/K.
    V : function
        Function of spatial coordinates (x, y, z) which is added to mu.

    Returns:
    --------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    """
    p = SimpleNamespace(t=t,
                        g=g,
                        mu_B=mu_B,
                        alpha=alpha,
                        B_x=B_x,
                        B_y=B_y,
                        B_z=B_z,
                        Delta=Delta,
                        mu=mu,
                        orbital=orbital,
                        V=V,
                        **kwargs)
    return p


def cylinder_sector(r1, r2=0, L=1, L0=0, phi=360, angle=0, a=10):
    """Returns the shape function and start coords.
    
    Parameters:
    -----------
    r1 : int
        Inner radius in nm.
    r2 : int
        Outer radius in nm.
    L : int
        Length of wire from L0 in nm, -1 if infinite in x-direction.
    L0 : int
        Start position in x.
    phi : int
        Coverage angle in degrees.
    angle : int
        Angle of tilting from top in degrees.
    a : int
        Discretization constant in nm.
    
    Returns:
    --------
    (shape_func, *(start_coords))
    """
    phi *= np.pi / 360
    angle *= np.pi / 180
    r1sq, r2sq = r1 ** 2, r2 ** 2
    def sector(pos):
        x, y, z = pos
        n = (y + 1j * z) * np.exp(1j * angle)
        y, z = n.real, n.imag
        rsq = y ** 2 + z ** 2
        
        shape_yz = r2sq <= rsq < r1sq and z >= np.cos(phi) * np.sqrt(rsq)
        return (shape_yz and L0 <= x < L) if L > 0 else shape_yz
    r_mid = (r1 + r2) / 2
    return sector, (L - a, r_mid * np.sin(angle), r_mid * np.cos(angle))


def discretized_hamiltonian(a=10):
    k_x, k_y, k_z = momentum_operators
    t, B_x, B_y, B_z, mu_B, Delta, mu, alpha, g, V = sympy.symbols('t B_x B_y B_z mu_B Delta mu alpha g V', real=True)
    t_interface = sympy.symbols('t_interface', real=True)
    k = sympy.sqrt(k_x**2+k_y**2+k_z**2)
    ham = ((t * k**2 - mu - V) * kr(s0, sz) +
               alpha * (k_y * kr(sx, sz) - k_x * kr(sy, sz)) +
               0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0) + B_z * kr(sz, s0)) +
               Delta * kr(s0, sx))
    args = dict(space_dependent={'V'}, lattice_constant=a, discrete_coordinates={'x', 'y', 'z'})
    tb_normal = Discretizer(ham.subs(Delta, 0), **args)
    tb_sc = Discretizer(ham.subs([(g, 0), (alpha, 0)]), **args)
    tb_interface = Discretizer(ham.subs(t, t_interface), **args)
    return tb_normal, tb_sc, tb_interface


def hoppingkind_in_shape(hop, shape, syst):
    """Returns an HoppingKind iterator for hoppings in shape."""
    def in_shape(site1, site2, shape):
        return shape[0](site1.pos) and shape[0](site2.pos)
    hoppingkind = kwant.HoppingKind(hop.delta, hop.family_a)(syst)
    return ((i, j) for (i, j) in hoppingkind if in_shape(i, j, shape))


def hoppingkind_at_interface(hop, shape1, shape2, syst):
    """Returns an HoppingKind iterator for hoppings at an interface between
       shape1 and shape2."""
    def at_interface(site1, site2, shape1, shape2):
        return ((shape1[0](site1.pos) and shape2[0](site2.pos)) or 
                (shape2[0](site1.pos) and shape1[0](site2.pos)))
    hoppingkind = kwant.HoppingKind(hop.delta, hop.family_a)(syst)
    return ((i, j) for (i, j) in hoppingkind if at_interface(i, j, shape1, shape2))


def peierls(val, ind, a, c=constants):
    def phase(s1, s2, p):
        x, y, z = s1.pos
        A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
        A_site *= a * 1e-18 * eV / hbar
        return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz

    def with_phase(s1, s2, p):
        return phase(s1, s2, p).dot(val(s1, s2, p)) if p.orbital else val(s1, s2, p)
    
    return with_phase

def make_3d_wire(a=10, L=50, r1=50, r2=70, phi=135, angle=0, disorder=False, 
                 with_vlead=True, with_leads=True, L_sc=1):
    """Creates a cylindrical 3D wire partially covered with a superconducting shell, 
    but without superconductor in the scattering region of length (L - 2*a*L_sc).
    
    Parameters:
    -----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire.
    r1 : int
        Radius of normal part of wire in nm.
    r2 : int
        Radius of superconductor in nm.
    phi : int
        Coverage angle of superconductor in degrees.
    angle : int
        Angle of tilting of superconductor from top in degrees.    
    disorder : bool
        When True syst requires 'disorder' and 'salt' aguments.
    with_vlead : bool
        If True a SelfEnergyLead with zero energy is added to a slice of the system.
    with_leads : bool
        If True it appends infinite leads with superconducting shell.
    L_sc : int
        Number of unit cells that has a superconducting shell. If the system has
        infinite leads, set L_sc=1.

    Returns:
    --------
    syst : kwant.builder.FiniteSystem
        The finilized system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.
    """
    tb_normal, tb_sc, tb_interface = discretized_hamiltonian(a)
    lat = tb_normal.lattice
    syst = kwant.Builder()
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    
    # Wire scattering region shapes
    shape_normal = cylinder_sector(r1=r1, angle=angle, L=L, a=a)
    # Superconductor slice in the beginning of the scattering region of L_sc unit cells
    shape_sc_start = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L=a*L_sc, a=a)
    # Superconductor slice in the end of the scattering region of L_sc unit cells
    shape_sc_end = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L0=L-a*L_sc, L=L, a=a)
    
    # Lead shapes
    shape_sc_lead = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L=-1, a=a)
    shape_normal_lead = cylinder_sector(r1=r1, angle=angle, L=-1, a=a)
    
    def onsite_dis(site, p):
        return p.disorder * (uniform(repr(site), repr(p.salt)) - 0.5) * np.eye(4)
    
    # Add onsite terms in the scattering region
    syst[lat.shape(*shape_normal)] = lambda s, p: tb_normal.onsite(s, p) + (onsite_dis(s, p) if disorder else 0)
    syst[lat.shape(*shape_sc_start)] = tb_sc.onsite
    syst[lat.shape(*shape_sc_end)] = tb_sc.onsite

    # Add onsite terms in the infinite lead
    lead[lat.shape(*shape_normal_lead)] = tb_normal.onsite
    lead[lat.shape(*shape_sc_lead)] = tb_sc.onsite

    for hop, val in tb_normal.hoppings.items():
        # Add hoppings in normal parts of wire and lead with Peierls substitution
        ind = np.argmax(hop.delta) # Index of direction of hopping
        syst[hoppingkind_in_shape(hop, shape_normal, syst)] = peierls(val, ind, a)
        lead[hoppingkind_in_shape(hop, shape_normal_lead, lead)] = peierls(val, ind, a)
        
    for hop, val in tb_sc.hoppings.items():
        # Add hoppings in superconducting parts of wire and lead
        syst[hoppingkind_in_shape(hop, shape_sc_start, syst)] = val
        syst[hoppingkind_in_shape(hop, shape_sc_end, syst)] = val
        lead[hoppingkind_in_shape(hop, shape_sc_lead, lead)] = val

    for hop, val in tb_interface.hoppings.items():
        # Add hoppings at the interface of superconducting parts and normal parts of wire and lead
        syst[hoppingkind_at_interface(hop, shape_sc_start, shape_normal, syst)] = val
        syst[hoppingkind_at_interface(hop, shape_sc_end, shape_normal, syst)] = val
        lead[hoppingkind_at_interface(hop, shape_sc_lead, shape_normal_lead, lead)] = val
        
    def cut(x_cut):
        # Take the sites at a cross section at x_cut
        sites = [lat(x, y, z) for x, y, z in (i.tag for i in syst.sites()) if x == x_cut]
        return sorted(sites, key=lambda s: s.pos[2] * 10000 + s.pos[1])
    
    # Define left and right cut in wire in the middle of the wire.
    l_cut = cut(L//a)
    r_cut = cut(L//a + 1)
    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
        
    if with_vlead:
        syst.leads.append(vlead)
    if with_leads:
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())
    syst = syst.finalized()

    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        # Function that returns the hopping matrix between the two cross sections
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


def matsubara_frequency(T, n):
    return (2*n + 1) * np.pi * T * 1j


def null_H(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return np.linalg.inv(gf.data[::2, ::2])


def gf_from_H_0(H_0, t):
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(T, H_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    I = 0
    for H_0 in H_0s:
        gf = gf_from_H_0(H_0, t - H12)
        dim = t.shape[0]
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I


def I_c(syst, hopping, p, T, matsfreqs=1):
    H_0s = [null_H(syst, p, T, n) for n in range(matsfreqs)]
    H12 = hopping(syst, [p])
    fun = lambda phase: -current_from_H_0(T, H_0s, H12, phase)
    opt = scipy.optimize.minimize_scalar(fun, bounds=(-np.pi, np.pi), method='bounded')
    phase = opt['x']
    current = -opt['fun']
    return phase, current


def save_data(fname, p, constants, Bs, matsfreqs, wire_params, T, I_cs, phases, **kwargs):
    
    if os.path.exists(fname):
        return "File already existed"
    
    def SimpleNamespace_save(p):
        """Removes functions from SimpleNamespace."""
        p_new = {key: val for key, val in p.__dict__.items() 
                 if not isinstance(val, types.FunctionType)}
        return SimpleNamespace(**p_new)
    
    data = {'p': SimpleNamespace_save(p),
            'constants': SimpleNamespace_save(constants),
            'B_xs': Bs,
            'matsfreqs': matsfreqs,
            'wire_params': wire_params,
            'T': T,
            'I_cs': I_cs,
            'phases': phases, **kwargs}

    dd.io.save(fname, data, compression='blosc')

In [ ]:
# Create system with infinite leads
wire_params = dict(a=10, L=60, r1=50, r2=70, phi=135, angle=0, 
                   disorder=False, with_vlead=True, with_leads=True, L_sc=1)
syst, hopping = make_3d_wire(**wire_params)
kwant.plot(syst)

In [ ]:
# Create system without leads
wire_params = dict(a=10, L=100, r1=50, r2=70, phi=135, angle=0, 
                   disorder=False, with_vlead=True, with_leads=False, L_sc=1)
syst, hopping = make_3d_wire(**wire_params)
kwant.plot(syst)

In [ ]:
# Create system without leads and bigger SC regions
wire_params = dict(a=10, L=100, r1=50, r2=70, phi=135, angle=0, 
                   disorder=False, with_vlead=True, with_leads=False, L_sc=3)
syst, hopping = make_3d_wire(**wire_params)
kwant.plot(syst)

# $T$ vs $I_c$ with disorder

In [ ]:
%%px --local
p = make_params(Delta=60, alpha=20, mu=15, B_x=0, B_y=0, t_interface=7/8*constants.t, disorder=None, salt=None)
wire_params = dict(a=10, L=600, r1=50, r2=70, phi=135, angle=0, disorder=True)
syst, hopping = make_3d_wire(**wire_params)

In [ ]:
Ts = np.arange(10, 160, 10) * 1e-3
salts = range(30)
disorders = [0, 10, 20, 30, 40]
I = lview.map_async(lambda x: I_c(syst, hopping, p.update(salt=x[0], disorder=x[1]), T=x[2]),
                    product(salts, disorders, Ts))

I.wait_interactive()

Ic = np.array(I.result()).reshape(len(salts), len(disorders), len(Ts), -1)

phases, I_cs = Ic.T

In [ ]:
save_data('T_vs_Ic_L_600_mu-15meV_new.h5', dview['p'][0], constants, None, 1, wire_params, Ts, I_cs, phases,
          disorders=disorders, salts=salts)

# Relative importance of Zeeman, orbital, and spin-orbit

In [ ]:
p = make_params(Delta=60, alpha=20, mu=0, B_x=0, B_y=0, t_interface=7/8*constants.t, disorder=None, salt=None)
wire_params = dict(a=10, L=600, r1=50, r2=70, phi=135, angle=0, disorder=True)
syst, hopping = make_3d_wire(**wire_params)
sorted(np.abs(kwant.physics.Bands(syst.leads[1], args=[p])(k=0)))[:13]

# Mean free path, $g \sim N_{ch}\lambda_{MFP}/L$

In [ ]:
%%px
def transmission(syst, p):
    smatrix = kwant.smatrix(syst, args=[p])
    return smatrix.num_propagating(0), smatrix.transmission(0, 1)

p = make_params(Delta=0, alpha=20, mu=6, B_x=0, B_y=0, B_z=0, t_interface=7/8*constants.t, disorder=None, salt=None)
wire_params = dict(a=10, L=None, r1=50, r2=70, phi=135, angle=0, disorder=True, with_vlead=False)

In [ ]:
Ls = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]
conductances = []
num_propagating = []
for L in Ls:
    print(L)
    dview.execute('wire_params["L"] = {}'.format(L))
    %px syst, hopping = make_3d_wire(**wire_params)
    salts = range(30)
    disorders = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
    conductance = lview.map_async(lambda x: transmission(syst, p.update(salt=x[0], disorder=x[1])), 
                        product(salts, disorders))
    conductance.wait_interactive()
    conductance = np.array(conductance.result()).reshape(len(salts), len(disorders), -1)
    num_propagating.append(conductance[:, :, 0])
    conductances.append(conductance[:, :, 1])

In [ ]:
save_data('data/mfp_mu-6meV.h5', dview['p'][0], dview['constants'][0], 0, 1, dview['wire_params'][0], 
          T=None, I_cs=None, phases=None, conductances=conductances,salts=salts,
          disorders=disorders, Ls=Ls, num_propagating=num_propagating)

In [ ]:
wire_params = dict(a=10, L=30, r1=50, r2=70, phi=135, angle=0, disorder=True, with_vlead=False)
syst, hopping = make_3d_wire(**wire_params)
kwant.plot(syst)

# Disorder

In [ ]:
%%px --local
p = make_params(Delta=60, alpha=20, mu=6, B_x=0, B_y=0, t_interface=7/8*constants.t, disorder=None, salt=None)
wire_params = dict(a=10, L=600, r1=50, r2=70, phi=135, angle=0, disorder=True)
T = 60e-3
syst, hopping = make_3d_wire(**wire_params)

In [ ]:
Bs = np.linspace(0, 1, 50)
salts = range(30)
disorders = [0, 10, 20, 30, 40]
I = lview.map_async(lambda x: I_c(syst, hopping, p.update(salt=x[0], disorder=x[1], B_x=x[2]), T), 
                    product(salts, disorders, Bs))

I.wait_interactive()

Ic = np.array(I.result()).reshape(len(salts), len(disorders), len(Bs), -1)

In [ ]:
curves = {}
for i, realisation in enumerate(Ic):
    for disorder, I_cs in zip(disorders, realisation):
        curves[(i, disorder)] = hv.Curve((Bs, (I_cs[:, 1])), kdims=[hv.Dimension('$B$', unit='$T$')], vdims=['$I_c$'])
hm = hv.HoloMap(curves, kdims=['salt', 'disorder'])

In [ ]:
hm

In [ ]:
collapsed = hm.collapse('salt', function=np.mean, spreadfn=np.std)
disorder = collapsed * collapsed.table().to.spread(['$B$', '$I_c$'], '$I_c$_std')
disorder

In [ ]:
%%output size=200 fig='svg'
hv.GridSpace(disorder)

In [ ]:
wire_params = dview['wire_params'][0]
p = dview['p'][0]
fname = 'L_{}-matfreqs_{}-a_{}-T_{}_disorder-mu-{}meV.h5'.format(wire_params['L'], 1, wire_params['a'], T, p.mu)
save_data(fname, p, constants, Bs, 1, wire_params, T, phases=Ic[..., 0], I_cs=Ic[..., 1], disorders=disorders, salts=salts)

# vary $L$, $n$, and $a$ without disorder:

In [ ]:
p = make_params(Delta=60, alpha=20, mu=15, B_x=None, B_y=0, t_interface=7/8*constants.t)
wire_params = dict(a=10, L=50, r1=50, r2=70, phi=135, angle=0)
Bs = np.linspace(0, 2, 100)
for T in [30e-3, 60e-3, 90e-3]:
    for wire_params['a'] in [5, 10]:
        for wire_params['L'] in [50, 100, 200, 400]:
            for matsfreqs in [1, 2, 3, 4]:
                print("T={}, a={}, L={}, matsfreq={}".format(T, wire_params['a'], wire_params['L'], matsfreqs))
                fname = 'L_{}-matfreqs_{}-a_{}-T_{}.h5'.format(wire_params['L'], matsfreqs, wire_params['a'], T)
                if not os.path.isfile(fname):
                    dview['T'] = T
                    dview['p'] = p
                    dview['wire_params'] = wire_params
                    dview['matsfreqs'] = matsfreqs
                    %px syst, hopping = make_3d_wire(**wire_params)
                    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x), T, matsfreqs), Bs)
                    I.wait_interactive()
                    phases, I_cs = np.array(I.result()).T
                    save_data(fname, p, constants, Bs, matsfreqs, wire_params, T, I_cs, phases)

In [ ]:
%%opts Path [aspect='square']
bands = kwant.physics.Bands(syst.leads[1], args=[p])
ks = np.linspace(-2, 2, 100)
en = np.array([bands(k=k) for k in ks])
hv.Path((ks, en))[:, -5:5]

In [ ]:
%%opts Overlay [show_legend=True] Curve [show_legend=True]
(hv.Curve((Bs, np.array([-I['fun'] for I in I_cs])), kdims=['$B_x$'], vdims=['$I_c$'], label='10 nm') 
 * hv.Curve((np.linspace(0, 1, 50), np.loadtxt('5nm.dat')), kdims=['$B_x$'], vdims=['$I_c$'], label='5 nm'))